In [ ]:
from itertools import pairwise
import pandas as pd
import numpy as np
from scipy.integrate import trapezoid as integrate_trapezoid
import matplotlib.pyplot as plt
from airfoil import (
    Airfoil,
    Hole,
    Hinge,
    WingSegment,
    Decomposer
)
from airfoil.cnc import MachineSetup, CNC
from airfoil.wing import (
    angle_degrees_to_slope,
    mirror,
    auto_piecewise,
    auto_interpolate,
    ellipse_quadrant,
    calculated_wing_cube_loading,
    create_airfoil_sampler,
)
from airfoil.util.linestring_helpers import resample_shapes
from airfoil.util.array_helpers import create_array_interpolator
import pyvista as pv
XPS_FOAM_DENSITY = 40 # kg/m**3

In [ ]:
elevator_sections_at = np.array([
    25,
    150,
])
elevator_section = mirror(create_airfoil_sampler(
    airfoil         = lambda _: Airfoil.from_naca_designation("0012",100),
    leading_edge    = auto_interpolate([
        [  0,   0],
        [ 25,   0],
        [150, -20],
    ]),
    dihedral        = lambda _: 0,
    chord           = auto_interpolate([
        [  0, 120],
        [ 25, 120],
        [150, 100],
    ]),
    washout         = lambda _: 0,
    rotation_center = lambda _: 0,
))
elevator_segments = [
    WingSegment(
        left   = elevator_section(a),
        right  = elevator_section(b),
        length = b-a,
    )
    for a,b
    in pairwise([-25]+elevator_sections_at.tolist())
]

In [ ]:
rudder_sections_at = np.array([
    0,
    25,
    150,
])
rudder_section = mirror(create_airfoil_sampler(
    airfoil         = lambda _: Airfoil.from_naca_designation("0012",100),
    leading_edge    = auto_interpolate([
        [  0,   0],
        [ 25,   0],
        [150, -20],
    ]),
    dihedral        = lambda _: 0,
    chord           = auto_interpolate([
        [  0, 120],
        [ 25, 120],
        [150, 100],
    ]),
    washout         = lambda _: 0,
    rotation_center = lambda _: 0,
))
rudder_segments = [
    WingSegment(
        left   = rudder_section(a),
        right  = rudder_section(b),
        length = b-a,
    )
    for a,b
    in pairwise(rudder_sections_at)
]

In [ ]:
wing_sections_at = np.array([
          25,
    350/4+25,      
    350/2+25,
    350  +25,
])
leading_edge = mirror(auto_piecewise([
    (      25, lambda x: 0),
    (350/2+25, lambda x: -x*angle_degrees_to_slope(4)),
    (  350+25, lambda x: -x*angle_degrees_to_slope(10)),
]))
trailing_edge = mirror(auto_piecewise([
    ( 25.0, lambda x: -200),
    #(350/2+25,, lambda x: -x*angle_degrees_to_slope( 10)),
    (350+25, lambda x: x*angle_degrees_to_slope( 2)),
]))
chord = lambda x: leading_edge(x)-trailing_edge(x)
dihedral = mirror(auto_piecewise([
    ( 25, lambda _: 0),
    #(350/2+25, lambda x: x * angle_degrees_to_slope( 1)),
    (350+25, lambda x: x * angle_degrees_to_slope( 5)),
]))
washout = mirror(auto_interpolate([
    [       0,  0],
    [350/2+25,  0],
    [350  +25, -2],
]))
hinge_line = mirror(lambda x: np.where(x>=350/2+25, trailing_edge(x)+chord(350/2+25)*0.3,np.nan))
spar_line  = mirror(lambda x: np.where(x<=350/4+25, -chord(0)*0.3,np.nan))
wing_airfoil = create_airfoil_sampler(
    airfoil         = lambda x: Airfoil.from_naca_designation("23012", 100),
    leading_edge    = leading_edge,
    dihedral        = dihedral,
    chord           = chord,
    washout         = washout,
    rotation_center = lambda x: chord(x)*0.25
)

wing_segments:list[WingSegment] = []
for sla,slb in pairwise([-25]+list(wing_sections_at)):
    af1 = wing_airfoil(sla)
    af2 = wing_airfoil(slb)
    mid = (sla+slb)/2
    # if not np.isnan(hinge_line(mid)):
    #     af1 = af1.with_hinge(Hinge([-hinge_line(sla),0],angle_deg=10,rotation_deg=0),upper_thickness=2)
    #     af2 = af2.with_hinge(Hinge([-hinge_line(slb),0],angle_deg=10,rotation_deg=0),upper_thickness=2)
    if not np.isnan(spar_line(mid)):
        af1 = af1.with_holes([Hole(8, (-spar_line(sla),8))])
        af2 = af2.with_holes([Hole(8, (-spar_line(slb),8))])
    wing_segments.append(WingSegment(
        af1,
        af2,
        slb-sla
    ))

In [ ]:
x_wing     = np.linspace(-(350+25),(350+25),400)
x_elevator = np.linspace(-150,150,400)
x_rudder = np.linspace(0,150,400)

fig, ax1 = plt.subplots(figsize=(5,8))
ax1.plot(x_wing,  leading_edge(x_wing))
ax1.plot(x_wing, trailing_edge(x_wing))
ax1.plot(x_wing, hinge_line(x_wing))
ax1.plot(x_wing, spar_line(x_wing))
ax1.set_aspect("equal")
for section in wing_sections_at:
    ax1.axvline(section,linestyle=":",c="r",linewidth=1)
    ax1.axvline(-section,linestyle=":",c="r",linewidth=1)

ax1.plot(x_elevator, -400 -np.array([*map(lambda x:elevator_section(x).points[:,0].min(),x_elevator)]))
ax1.plot(x_elevator, -400 -np.array([*map(lambda x:elevator_section(x).points[:,0].max(),x_elevator)]))
ax1.plot(rudder_section(0).points[:,1],-400-rudder_section(0).points[:,0])

fig, ax2 = plt.subplots(figsize=(5,8))
sar = (-wing_sections_at)[::-1].tolist()+wing_sections_at.tolist()
ax2.plot(sar, np.array([wing_airfoil(xi).points[:,1].max()/100*chord(xi)+dihedral(xi) for xi in sar]))
ax2.plot(sar, np.array([wing_airfoil(xi).points[:,1].min()/100*chord(xi)+dihedral(xi) for xi in sar]))
rl = np.array([*map(lambda x:rudder_section(x).points[:,1].min(),x_rudder)])
rr = np.array([*map(lambda x:rudder_section(x).points[:,1].max(),x_rudder)])
ax2.plot(rl,x_rudder-80)
ax2.plot(rr,x_rudder-80)
et = np.array([*map(lambda x:elevator_section(x).points[:,1].min(),x_elevator)])
eb = np.array([*map(lambda x:elevator_section(x).points[:,1].max(),x_elevator)])
ax2.plot(x_elevator,et-80)
ax2.plot(x_elevator,eb-80)
ax2.set_aspect("equal")

In [ ]:
wing_segments[1].right.plot_raw(show_hinge=True, show_holes=True)

In [ ]:
#wing_segments[2].right.plot_raw(show_hinge=True, show_holes=True)
for seg in wing_segments:
    de = Decomposer()
    fig,ax = plt.subplots(2, figsize=(8,3),sharex=True)
    seg.left.plot(ax=ax[0], decomposer=de)
    seg.right.plot(ax=ax[1], decomposer=de)

In [ ]:
for id, section in enumerate(wing_sections_at):
    pnts = wing_airfoil(section).points
    pnts = np.insert(pnts,0, section,axis=-1)
    pd.DataFrame(pnts, columns=["x","y","z"]).to_csv(f"./data/zippy/wing_tip_base_foil_{id}.csv",sep=",", header=True, index=False)

In [ ]:
pt = pv.Plotter()
wing_segment_meshes= WingSegment.to_meshes(wing_segments, add_mirrored=True, first_segment_is_central=True, share_decomposer=False)
for m in wing_segment_meshes:
    pt.add_mesh(
        m.rotate_x(-4).translate((0,0,60)),
        #pbr=True, 
        smooth_shading=True,
        split_sharp_edges=True,
        feature_angle=70,
        roughness=0.1,
        opacity=0.8,
    )
elevator_segment_meshes = WingSegment.to_meshes(elevator_segments, add_mirrored=True, first_segment_is_central=True)
for m in elevator_segment_meshes:
    pt.add_mesh(
        m.translate((0,400,0)),
        #opacity=0.8,
    )
rudder_segment_meshes = WingSegment.to_meshes(rudder_segments, first_segment_is_central=False)
for m in rudder_segment_meshes:
    pt.add_mesh(
        m.rotate_y(-90).translate((0,400,0)),
        #opacity=0.8,
    )
cl = 600
pt.add_mesh(pv.Cylinder((0,cl/2,0),direction=(0,1,0),radius=8,height=cl).translate((0,-180,0)))
pt.show()

In [ ]:
x = np.linspace(-wing_sections_at.max(),wing_sections_at.max(),400)
wing_volume = sum(item.volume for item in wing_segment_meshes)
wing_area = integrate_trapezoid(
    np.array([leading_edge(xi)-trailing_edge(xi) for xi in x]),
    x,
)
wing_area_m2      = wing_area/1000**2
wing_span         = np.max(x)-np.min(x)
mean_chord        = wing_area/wing_span
aspect_ratio      = wing_span/mean_chord
total_aircraft_mass_estimate     = 0.700
wing_cube_loading = calculated_wing_cube_loading(total_aircraft_mass_estimate,wing_area_m2)
wing_mass         = wing_volume/1000**3*XPS_FOAM_DENSITY *1000

print(f"""
{wing_span = :.1f}
{mean_chord = :.1f}
{aspect_ratio = :.1f}
{wing_area_m2 = :.3f}
{wing_mass = :.0f} g
{total_aircraft_mass_estimate = :.1f} kg
{wing_cube_loading = :.1f}
""")

In [ ]:
pt.export_gltf("./data/outputs/2025 07 20 Zippy.gltf")

In [ ]:
from typing import Literal
def cut_size(ws:WingSegment, excess:float=10, roundup:float=10):
    bs = ws.bounding_size()
    depth = bs[0]
    width = bs[2]
    return float(width), float(np.ceil(depth/roundup)*roundup+excess)
def cut_list(
        wss:list[WingSegment],
        label:str|None=None,
        mode:Literal["mirrored", "mirrored centered", "as is"]="mirrored centered",
        excess:float=10, roundup:float=5
    ):
    df = pd.DataFrame([cut_size(item, excess, roundup) for item in wss], columns=["width","depth"])
    match mode:
        case "mirrored":
            df = pd.concat([df,df])
        case "mirrored centered":
            df = pd.concat([df,df.iloc[1:]])
    res = df.groupby(["width","depth"]).size().rename("cut_list").to_frame()
    if label is not None:
        res["label"]=label
    return res
def cut_lists(ds:dict[str,tuple[list[WingSegment], Literal["mirrored", "mirrored centered", "as is"]]], excess:float=10, roundup:float=5):
    chunks = []
    for label, (segments, mode) in ds.items():
        chunks.append(cut_list(
            segments, 
            label=label, 
            mode=mode,
            excess=excess, 
            roundup=roundup
        ))
    return pd.concat(chunks).groupby(["width","depth"]).agg({"cut_list":"sum","label":lambda x: ", ".join(np.sort(np.unique(x)))})
cl = cut_lists({
    "wing": (wing_segments, "mirrored centered"),
    "elevator": (elevator_segments, "mirrored centered"),
    "rudder": (rudder_segments, "as is"),
})
cl

In [ ]:
# PLANE_SPACING = 245
# mirror = False
# segment_index = 0
# seg = wing_segments[segment_index]
# if mirror:
#     seg = seg.with_mirror()
# print(f"length:{seg.length} x depth:{np.ceil((seg.bounding_size()[0]+10)/10)*10}\nlenght/2={seg.length/2}")
# ms = MachineSetup(
#     seg,
#     foam_width  = seg.length,
#     foam_height = 30,
#     foam_depth = seg.bounding_size()[0]+10,
#     plane_spacing=PLANE_SPACING,
#     decomposer=Decomposer(buffer=0.5),
#     max_cut_speed_mm_s=110,
#     min_cut_speed_mm_s=105,
# ).with_recentered_part()
# instructions = ms.instructions(record_name=f"zippy_seg_wing_{segment_index}{'_mirror' if mirror else ''}")
# ms.plot((0,ms.foam_height,0,ms.foam_height))

In [ ]:
# PLANE_SPACING = 245
# mirror = True
# segment_index = 1
# seg = elevator_segments[segment_index]
# if mirror:
#     seg = seg.with_mirror()
# print(f"length:{seg.length} x depth:{np.ceil((seg.bounding_size()[0]+10)/10)*10}\nlenght/2={seg.length/2}")
# ms = MachineSetup(
#     seg,
#     foam_width  = seg.length,
#     foam_height = 30,
#     foam_depth = seg.bounding_size()[0]+10,
#     plane_spacing=PLANE_SPACING,
#     decomposer=Decomposer(buffer=0.5),
#     max_cut_speed_mm_s=115,
#     min_cut_speed_mm_s=105,
# ).with_recentered_part()
# instructions = ms.instructions(record_name=f"zippy_seg_elevator_{segment_index}{'_mirror' if mirror else ''}")
# ms.plot((0,ms.foam_height,0,ms.foam_height))

In [ ]:
# PLANE_SPACING = 245
# mirror = False
# segment_index = 1
# seg = rudder_segments[segment_index]
# if mirror:
#     seg = seg.with_mirror()
# print(f"length:{seg.length} x depth:{np.ceil((seg.bounding_size()[0]+10)/10)*10}\nlenght/2={seg.length/2}")
# ms = MachineSetup(
#     seg,
#     foam_width  = seg.length,
#     foam_height = 30,
#     foam_depth = seg.bounding_size()[0]+10,
#     plane_spacing=PLANE_SPACING,
#     decomposer=Decomposer(buffer=0.5),
#     max_cut_speed_mm_s=110,
#     min_cut_speed_mm_s=105,
# ).with_recentered_part()
# instructions = ms.instructions(record_name=f"zippy_seg_rudder_{segment_index}{'_mirror' if mirror else ''}")
# ms.plot((0,ms.foam_height,0,ms.foam_height))

In [ ]:
PLANE_SPACING = 245
mirror = True
segment_index = 0
seg = elevator_segments[segment_index]
if mirror:
    seg = seg.with_mirror()
print(f"length:{seg.length} x depth:{np.ceil((seg.bounding_size()[0]+10)/10)*10}\nlenght/2={seg.length/2}")
ms = MachineSetup(
    seg,
    foam_width  = seg.length,
    foam_height = 30,
    foam_depth = seg.bounding_size()[0],
    plane_spacing=PLANE_SPACING,
    decomposer=Decomposer(buffer=0.5),
    max_cut_speed_mm_s=115,
    min_cut_speed_mm_s=105,
).with_recentered_part()
instructions = ms.instructions(record_name=f"zippy_seg_rudder_elevator_interface_{segment_index}{'_mirror' if mirror else ''}")
ms.plot((0,ms.foam_height,0,ms.foam_height))

In [ ]:

#instructions[:,0]

In [ ]:
raise Exception("ready to cnc?")

In [ ]:
cnc = CNC()

In [ ]:
cnc.set_current(0)

In [ ]:
cnc.alarm_soft_reset()
cnc.alarm_clear()

In [ ]:
cnc.home()
cnc.set_position(0,0,0,0)

In [ ]:
# manually align
cnc.relative()
cnc.travel(
    x=0,y=-2,
    z=0,a=-2,
)

In [ ]:
def set_current(current:float):
    MAX = 4
    MIN = 0
    if current<=MIN:
        print("M3 S5")
        return cnc.writeln("M3 S5")
    
    portion = (current-MIN)/(MAX-MIN)
    pwm_percent = (portion*0.8+0.1)*100
    print(f"M3 S{pwm_percent:.0f}")
    return cnc.writeln(f"M3 S{pwm_percent:.1f}")

In [ ]:
# main cut
cnc.set_position(
    0,ms.foam_height,
    0,ms.foam_height,
)
cnc.absolute()
set_current(1.85)
cnc.send_g1_commands(instructions)
set_current(0)

In [ ]:
set_current(1.85)

In [ ]:
set_current(0)

In [ ]:
cnc.relative()
cnc.travel(
    0,15,
    0,15,
)

In [ ]:
cnc.absolute()
cnc.travel(
    0,ms.foam_height,
    0,ms.foam_height,
)

In [ ]:
# all clear?
# travel home
cnc.absolute()
cnc.travel(0,0,0,0)